# ダーティデータに対する処理

In [1]:
import pandas as pd
import polars as pl
import numpy as np

## Case1: 半角スペースが入ったフィールドを含むCSV
下記のように最初の4フィールドは普通の値だが、5フィールド目はスペースが入った1つの値となっており、  
これらをスペースで区切ってフィールドを分け、それぞれのフィールドに対して個別に処理したい場合を考える。  
ただし、個別に処理したのち、元通りの形式に戻す必要があるものとする。  
(カンマ区切りのまま読み込んでもよいが、5フィールド目の特定位置の値を編集するのが面倒なので結局分けることになる。)  
5つ目のフィールドは元々データ長が左から8,8,8,11,3となっていたもので、右詰めでスペースでパディングされているものとする。  

例）
A,123456,111,,     Red    Blue  Yellow   20190203  1

上記のデータに対して以下の操作を行う（操作内容は任意）  
* 5フィールド目の左から3つ目の列におけるYellowをBrownに置き換える
* 3フィールド目の111を222に置き換える
* 2行目の値を10行分複製する。この時、5フィールド目の一番右の値は連番にする。

### データ読み込み

In [2]:
# polarsではregexによる分割はできないので、pandasを用いる。
# engine='python'を指定しないと、regexを認識できないという警告が出る。
col_names = [f'col_{i}' for i in range(1, 11)]
data_type = {
    'col_1': str,
    'col_2': str,
    'col_3': str,
    'col_4': str,
    'col_5': str,
    'col_6': str,
    'col_7': str,
    'col_8': str,
    'col_9': str,
    'col_10': int,
}
# ,|\s+でカンマまたは１つ以上のスペースで区切る
df_pd = pd.read_csv('input/dirty_data_1.csv', sep=',|\s+', index_col=None, engine='python', names=col_names, dtype=data_type)
df_pd

,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10
0,A,123456,111,NaN,NaN,Red,Blue,Yellow,20190203,1
1,B,234567,111,NaN,NaN,Blue,Red,Yellow,20190204,2
2,B,345678,111,NaN,NaN,Red,Yellow,Yellow,20190205,3


In [3]:
# 不要なカラム削除
df_pd.drop(['col_5'], axis=1, inplace=True)
df_pd

,col_1,col_2,col_3,col_4,col_6,col_7,col_8,col_9,col_10
0,A,123456,111,NaN,Red,Blue,Yellow,20190203,1
1,B,234567,111,NaN,Blue,Red,Yellow,20190204,2
2,B,345678,111,NaN,Red,Yellow,Yellow,20190205,3


### データに対する編集操作（任意）
* col_8のYellowをBrownに置き換える
* col_3の111を222に置き換える
* 2行目の値を10行分複製する。この時、col_10の値は連番にする。

In [4]:
# Dataframeの操作はpolarsでやる
df_pl = pl.from_pandas(df_pd)
target_col1 = 'col_8'
target_col2 = 'col_2'

df_pl = df_pl.with_columns(
    pl.when(pl.col(target_col1) == 'Yellow').then('Brown').otherwise(pl.col(target_col1)).alias(target_col1),
    pl.when(pl.col(target_col2) == '111').then('222').otherwise(pl.col(target_col2)).alias(target_col2),
)

In [5]:
# 2行目の複製
replicate_row = df_pl[1]
df_result = df_pl
for i in range(1, 11):
    df_result = pl.concat([df_result, replicate_row])


# col_9を連番にする
df_result = df_result.with_row_count(offset=1).with_columns(
    (pl.col('row_nr')).cast(pl.Utf8).alias('col_10')
).drop('row_nr')

In [6]:
df_result

col_1,col_2,col_3,col_4,col_6,col_7,col_8,col_9,col_10
str,str,str,null,str,str,str,str,str
"""A""","""123456""","""111""",null,"""Red""","""Blue""","""Brown""","""20190203""","""1"""
"""B""","""234567""","""111""",null,"""Blue""","""Red""","""Brown""","""20190204""","""2"""
"""B""","""345678""","""111""",null,"""Red""","""Yellow""","""Brown""","""20190205""","""3"""
"""B""","""234567""","""111""",null,"""Blue""","""Red""","""Brown""","""20190204""","""4"""
"""B""","""234567""","""111""",null,"""Blue""","""Red""","""Brown""","""20190204""","""5"""
"""B""","""234567""","""111""",null,"""Blue""","""Red""","""Brown""","""20190204""","""6"""
"""B""","""234567""","""111""",null,"""Blue""","""Red""","""Brown""","""20190204""","""7"""
"""B""","""234567""","""111""",null,"""Blue""","""Red""","""Brown""","""20190204""","""8"""
"""B""","""234567""","""111""",null,"""Blue""","""Red""","""Brown""","""20190204""","""9"""


### CSVへの書き出し
元のフォーマットを維持して書きだすためにcol_6～col_10は右詰め・スペースでパディングする必要がある。

In [7]:
# スペースでパディングする用の辞書
data_len = {
    'col_6': 8,
    'col_7': 8,
    'col_8': 8,
    'col_9': 11,
    'col_10': 3
}

In [8]:
# 各列のデータ長になるようにスペースでパディング
for key, val in data_len.items():
    df_result = df_result.with_columns(
        pl.col(key).str.rjust(val, ' ').alias(key)
    )

In [9]:
df_result.head(3)

col_1,col_2,col_3,col_4,col_6,col_7,col_8,col_9,col_10
str,str,str,null,str,str,str,str,str
"""A""","""123456""","""111""",null,""" Red""",""" Blue""",""" Brown""",""" 20190203""",""" 1"""
"""B""","""234567""","""111""",null,""" Blue""",""" Red""",""" Brown""",""" 20190204""",""" 2"""
"""B""","""345678""","""111""",null,""" Red""",""" Yellow""",""" Brown""",""" 20190205""",""" 3"""


In [10]:
len(df_result['col_10'][0])

3

パッと見てパディングされていないように見えるが、データ長は確かに合っている。  
気になるようであればスペース等でパディングしておいて、csv側で置換するのもあり。

In [11]:
# 分割されたカラムのまま出力してしまうとカンマが入るので、元々のスペースが入った形に戻す。
merge_cols = [f'col_{i}' for i in range(6, 11)]
expr = ''
for col in merge_cols:
    # カラム結合用のexpr
    expr += pl.col(col)


# 元に戻す＆分割されたカラムは削除
df_result_csv = df_result.with_columns(
    expr.alias('dirty_cols')
).drop(merge_cols)


In [13]:
# CSVへ書き出し
df_result_csv.write_csv('output/dirty_data_1_mod.csv', has_header=False)